#### Code By: Ryan Abramowitz

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.patches import Rectangle
import numpy as np
import random
import scipy
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams["font.family"] = "Arial"
import gc

In [ ]:
# Like 5 mins

outdir = 'output/'
os.makedirs(outdir,exist_ok=True)
workdir = 'output/tmp/'
os.makedirs(workdir,exist_ok=True)

phastcon = 'hg38.phastCons20way.bw'
background = 'background.bed'
output_background = workdir+background[:background.rfind('.')]+'.conservation.bed'
if not os.path.isfile(output_background):
    command1 = f"bedtools random -g hg38.chrom.sizes -l 401 -seed 1 > {background}"
    os.system(command1)
    command3 = f"./bigWigAverageOverBed {phastcon} {background} {output_background}"
    os.system(command3)
    print('1: Ran something 1ce!')
df = pd.read_csv('final_peakset_anno.csv')
wheredict = {
    'exon' : df['Annotation'].str.startswith('exon'),
    'promoter' : df['Annotation'].str.startswith('promoter-TSS'),
    'intron' : df['Annotation'].str.startswith('intron'),
    'distal' : np.array([df['Annotation'].str.startswith(i) == False for i in  ['promoter-TSS','exon','intron','TTS', "3'", "5'"]]).all(axis=0),
}
df['Start'] -= 1
xisin = df[['Chr','Start','End']].astype(str).apply(lambda x: '\t'.join(x),axis=1)
for subtype in wheredict.keys():
    if not os.path.isfile(workdir + subtype + '-' + 'consensus_40k_for_maelstrom_ATAC_H3K27ac_bed_file.conservation.bed'):
        for input in ['consensus_40k_for_maelstrom.bed','consensus_40k_for_maelstrom_ATAC_H3K27ac.bed']:
            true_input = workdir + subtype + '-' + input[:input.find('.')] + '_bed_file.bed'
            output = true_input[:true_input.rfind('.')]+'.conservation.bed'
            with open(true_input,'w+') as f:
                x = pd.read_csv(input,sep='\t',index_col=0).index.str.replace(':','\t').str.replace('-','\t')
                allowed_peaks = xisin.loc[wheredict[subtype]]
                x = x[x.isin(allowed_peaks)]
                f.write('\n'.join(x + '\tPeak' + pd.Index(range(len(x))).astype(str)))
            # command1 = f"bedtools shuffle -i {true_input} -g hg38.chrom.sizes -seed 1 > {background}"
            command2 = f"./bigWigAverageOverBed {phastcon} {true_input} {output}"
            os.system(command2)
            print('2: Ran something 1ce!')

    if not os.path.isfile(f'{workdir}{subtype}-consensus_40k_for_maelstrom.conserved_peaks.bed'):
        for input in ['consensus_40k_for_maelstrom_ATAC_H3K27ac','consensus_40k_for_maelstrom']:
            df = pd.read_csv(f'{workdir}{subtype}-{input}_bed_file.conservation.bed',sep='\t',names=['name','size','covered','sum','mean0','mean'])
            names = df.loc[df['mean'] > 0.5]['name']
            locs = pd.read_csv(f'{workdir}{subtype}-{input}_bed_file.bed',sep='\t',names=['Chr','Start','End','Name'],index_col=3)
            conserved = locs.loc[names]
            conserved.to_csv(f'{workdir}{subtype}-{input}.conserved_peaks.bed',sep='\t',header=False,index=False)
            print('3: Ran something 1ce!')
if not os.path.isfile(f'{workdir}promoter-consensus_40k_for_maelstrom.conserved_peaks.GO_Biological_Process.csv'):
    os.system("Rscript get_cons.R")
gc.collect()



colors = [
    'b',
    'plum',
    'orange',
    'firebrick',
    'k',
]
nbins = 5
binEdges = np.array([i/nbins for i in range(nbins+1)])
bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
binsize = 0.5
cats = len(wheredict.keys())+1
width = 1/cats*binsize/nbins
binstart = width/binsize/2
xstart = bincenters-binstart*2
for input in ['consensus_40k_for_maelstrom','consensus_40k_for_maelstrom_ATAC_H3K27ac']:
    for loc,subtype in enumerate(wheredict.keys()):
        df = pd.read_csv(workdir + subtype + '-' + input + '_bed_file.conservation.bed',sep='\t',names=['name','size','covered','sum','mean0','mean'])
        yvals = np.array(df['mean'])
        print(subtype,'has',(yvals >= 0.5).sum(),'conserved and',(yvals < 0.5).sum(),'not conserved for',input)
        ytrue,_ = np.histogram(yvals,bins=binEdges)
        x = xstart + loc*width
        yerr = np.sqrt(ytrue*(1-ytrue/ytrue.sum()))*100/ytrue.sum()
        plt.bar(x, ytrue*100/ytrue.sum(), width=width, color=colors[loc], yerr=yerr, edgecolor = 'k',capsize=5)
    loc += 1
    ctrl = pd.read_csv(output_background,sep='\t',names=['name','size','covered','sum','mean0','mean'])
    cvals = np.array(ctrl['mean'])
    ctrltrue,_ = np.histogram(cvals,bins=binEdges)
    x = xstart + loc*width
    yerr = np.sqrt(ctrltrue*(1-ctrltrue/ctrltrue.sum()))*100/ctrltrue.sum()
    plt.bar(x, ctrltrue*100/ctrltrue.sum(), width=width, color=colors[loc], yerr=yerr, edgecolor = 'k',capsize=5)

    handles = [Rectangle((0,0),1,1,color=c,ec="k") for c in colors]
    labels= list(wheredict.keys()) + ["background"]
    plt.legend(handles, labels,)
    plt.yscale('log')
    plt.ylabel('Percentage (%)')
    plt.xlabel('Mean Concervation')
    plt.title(input)
    plt.savefig(outdir+input+'.conservation.pdf')
    plt.show()


colors = [
    'b',
    'plum',
    'orange',
    'firebrick',
]
nbins = 5
binEdges = np.array([i/nbins for i in range(nbins+1)])
bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
binsize = 0.5
cats = len(wheredict.keys())
width = 1/cats*binsize/nbins
binstart = width/binsize/2
xstart = bincenters-binstart*2
for input in ['consensus_40k_for_maelstrom','consensus_40k_for_maelstrom_ATAC_H3K27ac']:
    ctrl = pd.read_csv(output_background,sep='\t',names=['name','size','covered','sum','mean0','mean'])
    cvals = np.array(ctrl['mean'])
    ctrltrue,_ = np.histogram(cvals,bins=binEdges)
    ctrlerr = np.sqrt(ctrltrue*(1-ctrltrue/ctrltrue.sum()))*100/ctrltrue.sum()
    ctrltrue = ctrltrue*100/ctrltrue.sum()
    for loc,subtype in enumerate(wheredict.keys()):
        df = pd.read_csv(workdir + subtype + '-' + input + '_bed_file.conservation.bed',sep='\t',names=['name','size','covered','sum','mean0','mean'])
        yvals = np.array(df['mean'])
        print(subtype,'has',(yvals >= 0.5).sum(),'conserved and',(yvals < 0.5).sum(),'not conserved for',input)
        ytrue,_ = np.histogram(yvals,bins=binEdges)
        x = xstart + loc*width
        yerr = np.sqrt(ytrue*(1-ytrue/ytrue.sum()))*100/ytrue.sum()
        ytrue = ytrue*100/ytrue.sum()
        rtrue = ytrue/ctrltrue
        rerr = rtrue * np.sqrt((yerr/ytrue)**2 + (ctrlerr/ctrltrue)**2)
        plt.bar(x, rtrue, width=width, color=colors[loc], yerr=rerr, edgecolor = 'k',capsize=5)

    handles = [Rectangle((0,0),1,1,color=c,ec="k") for c in colors]
    labels= list(wheredict.keys())
    plt.legend(handles, labels,)
    plt.yscale('log')
    plt.ylabel('Fold change over background')
    plt.xlabel('Mean Concervation')
    plt.title(input)
    plt.savefig(outdir+input+'.fold_change.conservation.pdf')
    plt.show()


In [ ]:
# ~35 mins


outdir = 'output/'
os.makedirs(outdir,exist_ok=True)
workdir = 'output/tmp/'
os.makedirs(workdir,exist_ok=True)


nexpand = 10_000
if not os.path.isfile(workdir + 'background.bed'):
    for input in ['consensus_40k_for_maelstrom.bed','consensus_40k_for_maelstrom_ATAC_H3K27ac.bed','background.bed']:
        true_input = workdir + 'tmp-' + input
        finale = workdir + input
        with open(true_input,'w+') as f:
            if input != 'background.bed':
                x = pd.read_csv(input,sep='\t',index_col=0).index.str.replace(':','\t').str.replace('-','\t')
            else:
                x = pd.read_csv(input,sep='\t',index_col=None,names=[0,1,2,3,4,5])[[0,1,2,]].astype(str).apply(lambda x: '\t'.join(x),axis=1)
            f.write('\n'.join(x + '\tPeak' + pd.Index(range(len(x))).astype(str)))
        os.system(f'bedtools slop -i {true_input} -g hg38.chrom.sizes -b {nexpand} > {finale}')
        del x
        gc.collect()
        print('Did a thing!')
newlen = 401+nexpand*2
if not os.path.isfile(workdir + 'Background.gzip'):
    # https://github.com/deeptools/pyBigWig
    import pyBigWig
    bw = pyBigWig.open("hg38.phastCons20way.bw")
    for file,label in zip(['consensus_40k_for_maelstrom_ATAC_H3K27ac.bed','consensus_40k_for_maelstrom.bed','background.bed'],['ATAC+H3K27ac','ATAC','Background']):
        with open(workdir+file) as f:
            f.readline()
            scorelist = 0
            elems = 0
            for l in f:
                l = l.split('\t')
                l = np.array(bw.values(l[0], int(l[1]), int(l[2]))[:newlen])
                if not np.isnan(l).any():
                    elems += 1
                    scorelist += l
            scorelist /= elems
            np.savetxt(workdir+label+'.gzip',scorelist)
            del scorelist
        print('Did a thing!')
x = np.array([i-newlen/2 for i in range(newlen)])

for wid in [2_500,1e99]:
    colors = ['darkgreen','purple','k']
    for label,color in zip(['ATAC+H3K27ac','ATAC','Background'],colors):
        scorelist = np.loadtxt(workdir+label+'.gzip')
        plt.plot(x[np.abs(x) <= wid],scorelist[np.abs(x) <= wid],color=color,label=label)
        del scorelist
        gc.collect()
    print('Did a thing!')
    plt.legend(loc='best')
    plt.ylabel('Base Conservation')
    plt.xlabel('Peak Nucleotide')
    if wid > 10**9:
        plt.savefig(outdir+'ATAC_H3K27ac_base_conservation.pdf')
    else:
        plt.savefig(outdir+'small='+str(wid)+'_ATAC_H3K27ac_base_conservation.pdf')
    plt.show()


    colors = ['purple','k']
    for label,color in zip(['ATAC','Background'],colors):
        scorelist = np.loadtxt(workdir+label+'.gzip')
        plt.plot(x[np.abs(x) <= wid],scorelist[np.abs(x) <= wid],color=color,label=label)
        del scorelist
        gc.collect()
    print('Did a thing!')
    plt.legend(loc='best')
    plt.ylabel('Base Conservation')
    plt.xlabel('Peak Nucleotide')
    if wid > 10**9:
        plt.savefig(outdir+'ATAC_base_conservation.pdf')
    else:
        plt.savefig(outdir+'small='+str(wid)+'_ATAC_base_conservation.pdf')
    plt.show()

